In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!pip install barbar

In [4]:

from IPython.core.display import Video, Image, HTML
import time
import copy
import pickle
from barbar import Bar

import numpy as np 
import pandas as pd 
import scipy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
import cv2
%matplotlib inline
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import transforms

from tqdm import tqdm
from pathlib import Path
import gc
RANDOMSTATE = 0
import os

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [6]:
datasetPath = Path('/content/gdrive/MyDrive/Traffic_congestion/Dataset/trafficnet_dataset_v1/train')
congested = datasetPath/'dense_traffic'
sparse = datasetPath/'sparse_traffic'
df = pd.DataFrame()

In [7]:
for f in os.listdir(congested):
    if os.path.isfile(os.path.join(congested, f)):
        df = df.append({
                'image': str(os.path.join(congested,f)),
                'congestion': 1
            }, ignore_index=True)

for f in os.listdir(sparse):
    if os.path.isfile(os.path.join(sparse, f)):
        df = df.append({
                'image': str(os.path.join(sparse,f)),
                'congestion': 0
            }, ignore_index=True)

df = df.sample(frac=1).reset_index(drop=True)
df.head()


,image,congestion
0,/content/gdrive/MyDrive/Traffic_congestion/Dat...,0.0
1,/content/gdrive/MyDrive/Traffic_congestion/Dat...,0.0
2,/content/gdrive/MyDrive/Traffic_congestion/Dat...,0.0
3,/content/gdrive/MyDrive/Traffic_congestion/Dat...,0.0
4,/content/gdrive/MyDrive/Traffic_congestion/Dat...,0.0


In [8]:
class Dataset(Dataset):

    def __init__(self, dataFrame):
        self.dataFrame = dataFrame
        
        self.transformations = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    
    def __getitem__(self, key):
        if isinstance(key, slice):
            raise NotImplementedError('slicing is not supported')
        
        row = self.dataFrame.iloc[key]
        image = self.transformations(Image.open(row['image']))
        label = torch.tensor([row['congestion']], dtype=float)
        return image,label
    
    def __len__(self):
        return len(self.dataFrame.index)

In [9]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=1511b796c23c86c63aa662d10ae22e2f174282020198b0513a02e5c6c035a418
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [10]:
from efficientnet_pytorch import EfficientNet

In [11]:
class TransferLearningModel(nn.Module):
    @staticmethod
    def model(version):
        basemodel = EfficientNet.from_pretrained(version)

        
        for param in basemodel.parameters():
            param.requires_grad = False
        num_ftrs = basemodel._fc.in_features

        basemodel._fc = nn.Linear(num_ftrs, 1)

        return basemodel

In [12]:
def prepare_data(DF):
    trainDF, validateDF = train_test_split(DF, test_size=0.15, random_state=RANDOMSTATE)
    train_set = Dataset(trainDF)
    validate_set = Dataset(validateDF)
    
    return train_set, validate_set

In [13]:
def load_ckpt(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    checkpoint = torch.load(checkpoint_fpath)    
    model.load_state_dict(checkpoint['model_state_dict'])
    
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])     
    return model, optimizer, checkpoint['epoch']

def save_checkpoint(state, filename):
    print ("=> Saving a new best")
    torch.save(state, filename)  # save checkpoint

def train_model(model, 
                start_epoch, 
                criterion, 
                optimizer,  
                num_epochs):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(start_epoch, start_epoch+num_epochs+1):
        print('Epoch {}/{}'.format(epoch, start_epoch+num_epochs))
        print('-' * 10)

        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train() 
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

            
            for idx,(inputs, labels) in enumerate(Bar(dataloaders[phase])):
                inputs = inputs.to(device)
                labels = torch.as_tensor(labels, dtype = torch.float)
                labels = labels.to(device)

                
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    m = nn.Sigmoid()

                    preds = torch.sigmoid(outputs)
                    preds = preds>0.5
    
                    loss = criterion(m(outputs).to(device), labels)

                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                save_checkpoint(state={   
                                    'epoch': epoch,
                                    'state_dict': model.state_dict(),
                                    'best_accuracy': best_acc,
                                    'optimizer_state_dict':optimizer.state_dict()
                                },filename='ckpt_epoch_{}.pt'.format(epoch))

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    
    model.load_state_dict(best_model_wts)
    return model, optimizer, epoch_loss

In [16]:
EPOCHS = 1
start_epoch = 1
RETRAIN=False

trainDF, validateDF = prepare_data(df)

# declare the dataloader from the dataframe
dataloaders = {'train': DataLoader(trainDF, batch_size=32, shuffle=True, num_workers=1) ,
                'val':DataLoader(validateDF, batch_size=32, num_workers=1)
                }

dataset_sizes = {'train': len(trainDF),'val':len(validateDF)}

#initialize the model and put it acceleration(if available)
model = TransferLearningModel.model(version='efficientnet-b2')
model = model.to(device)

criterion = nn.BCELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

if RETRAIN==True:
    # load the saved checkpoint
    model, optimizer, start_epoch = load_ckpt(args['model'], model, optimizer)
    print('Checkpoint Loaded...')

Loaded pretrained weights for efficientnet-b2


In [17]:
model, optimizer, loss = train_model(model=model, 
                    start_epoch=start_epoch,
                    criterion=criterion, 
                    optimizer=optimizer, 
                    num_epochs=EPOCHS)

Epoch 1/2
----------
1870/1870: [===============================>] - ETA 21.5s
train Loss: 0.6298 Acc: 0.7053
330/330: [=============================>..] - ETA 8.7s
val Loss: 0.5884 Acc: 0.7667
=> Saving a new best

Epoch 2/2
----------
1870/1870: [===============================>] - ETA 5.8s
train Loss: 0.5293 Acc: 0.8353
330/330: [=============================>..] - ETA 4.6s
val Loss: 0.5025 Acc: 0.8424
=> Saving a new best

Training complete in 11m 35s
Best val Acc: 0.842424


In [18]:
torch.save({
            'epoch': EPOCHS,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, 'efficientnet_v1.pt')

In [19]:
model = TransferLearningModel.model(version='efficientnet-b2')
model.load_state_dict(torch.load('/content/efficientnet_v1.pt', map_location=device)['model_state_dict'], strict=False)

model = model.to(device)

model.eval()

Loaded pretrained weights for efficientnet-b2


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    

In [26]:
transformations = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        ])

In [27]:
%%HTML
<video width="320" height="240" controls>
  <source src="/content/gdrive/MyDrive/Traffic_congestion/test.mp4" type="video/mp4">
</video>

In [32]:
video_path= '/content/gdrive/MyDrive/Traffic_congestion/test.mp4'

In [35]:
def traffic_congestion_prediction(video_path):
  font = cv2.FONT_HERSHEY_SIMPLEX
  labels = ['Sparse', 'Congested']

  labelColor = [(0, 255, 0),(0, 0, 255)]

  vs = cv2.VideoCapture(video_path)

  codec = "MJPG"
  fps = 30
  capture_size = (int(vs.get(3)), int(vs.get(4)))
  fourcc = cv2.VideoWriter_fourcc(*codec)
  writer = cv2.VideoWriter('output.avi', fourcc, fps,capture_size)

  num_frames = int(vs.get(cv2.CAP_PROP_FRAME_COUNT))
  process_this_frame = True
  for i in tqdm(range(num_frames)):
      
      _ , frame = vs.read()
      if process_this_frame:
          outputs = model(transformations(frame).unsqueeze(0).to(device))
          predicted = torch.sigmoid(outputs)
          predicted = predicted>0.5

      process_this_frame = not process_this_frame

      
      textSize = cv2.getTextSize(labels[predicted], font, 1, 2)[0]

      
      cv2.putText(frame,
                  'Traffic Status:{}'.format(labels[predicted]),
                  (0, 80),
                  font, 1, labelColor[predicted], 2)   


      if writer is not None:
          writer.write(frame)


  vs.release()
  return labels[predicted]